In [1]:
from lib import dataloading as dl
from lib import tokenizer as tk
import pandas as pd
from pathlib import Path
from tqdm import tqdm

In [2]:
english_train_df = pd.read_csv('D:\Dropbox\Bachelorarbeit\Datasets\English\Train\english_train.csv', dtype={5: str})
english_train_df = dl.clear_non_UPOS_tags(english_train_df)
english_test_df = pd.read_csv('D:\Dropbox\Bachelorarbeit\Datasets\English\Test\english_test.csv', dtype={5: str})
english_test_df = dl.clear_non_UPOS_tags(english_test_df)

turkish_train_df = pd.read_csv('D:\Dropbox\Bachelorarbeit\Datasets\Turkish\Train\\turkish_train.csv', dtype={5: str})
turkish_train_df = dl.clear_non_UPOS_tags(turkish_train_df)
turkish_test_df = pd.read_csv('D:\Dropbox\Bachelorarbeit\Datasets\Turkish\Test\\turkish_test.csv', dtype={5: str})
turkish_test_df = dl.clear_non_UPOS_tags(turkish_test_df)

english_dir = Path("D:/Dropbox/Bachelorarbeit/Tokenization by Word Tag/Tokenizers/English")
english_dir.mkdir(parents=True, exist_ok=True)

turkish_dir = Path("D:/Dropbox/Bachelorarbeit/Tokenization by Word Tag/Tokenizers/Turkish")
turkish_dir.mkdir(parents=True, exist_ok=True)

Dropped 17911 rows with non-UPOS tags 
Tags dropped: ['_']
Dropped 4791 rows with non-UPOS tags 
Tags dropped: ['_']
Dropped 4541 rows with non-UPOS tags 
Tags dropped: ['_']
Dropped 1209 rows with non-UPOS tags 
Tags dropped: ['_']


In [3]:
vocab_sizes = [4096, 8192, 16384]
algortihms = ["BPE", "WordPiece", "Unigram"]

In [4]:
with tqdm(total=len(vocab_sizes) * len(algortihms), desc="Training Tokenizers") as pbar:
    for vocab_size in vocab_sizes:
        for algorithm in algortihms:
            pbar.set_description(f"Training tokenizer with vocab_size {vocab_size} using the {algorithm} algorithm")
            path = english_dir / f"upos_{algorithm.lower()}_{vocab_size}_tokenizer.json"
            tokenizer = tk.train_and_merge_tokenizers(
                english_train_df,
                tokenizer_algorithm=algorithm,
                vocab_size=vocab_size,
                allocation="proportional",
                save_path=str(path)
            )
            pbar.update(1)

Training tokenizer with vocab_size 16384 using the Unigram algorithm: 100%|██████████| 9/9 [00:37<00:00,  4.18s/it]  


In [5]:
with tqdm(total=len(vocab_sizes) * len(algortihms), desc="Training Tokenizers") as pbar:
    for vocab_size in vocab_sizes:
        for algorithm in algortihms:
            pbar.set_description(f"Training tokenizer with vocab_size {vocab_size} using the {algorithm} algorithm")
            path = turkish_dir / f"upos_{algorithm.lower()}_{vocab_size}_tokenizer.json"
            tokenizer = tk.train_and_merge_tokenizers(
                turkish_train_df,
                tokenizer_algorithm=algorithm,
                vocab_size=vocab_size,
                allocation="proportional",
                save_path=str(path)
            )
            pbar.update(1)

Training tokenizer with vocab_size 16384 using the Unigram algorithm: 100%|██████████| 9/9 [01:01<00:00,  6.88s/it]  
